In [2]:
!pip install -U langchain-text-splitters
!pip install -U langchain-community langchain-text-splitters langchain
!pip install pypdf
!pip install -U langchain-huggingface
!pip install -qU langchain-chroma
!pip install -qU langchain
!pip install -qU langchain-groq
!pip install sentence-transformers
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.7
    Uninstalling l

In [66]:
from langchain_groq import ChatGroq
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import shutil
import gradio as gr
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import re
import gc

In [67]:
llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    temperature=0.5
)

In [68]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer',
    input_key='question'
)

memory.clear()
# Définir les instructions (Prompt)
instruction_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "Tu es un expert en analyse de documents. Ton rôle est de fournir des résumés clairs, "
        "précis et structurés à partir du contexte fourni ci-dessous.\n\n"
        "CONTEXTE :\n{context}\n\n"
        "DIRECTIVES :\n"
        "1. Agis comme un analyste senior : synthétise les points clés, les noms, les dates et les montants importants.\n"
        "2. Si l'information demandée n'est pas présente dans le contexte, dis simplement que tu ne sais pas. N'invente jamais d'informations.\n"
        "3. Ignore les éléments répétitifs de mise en page (comme les adresses de pied de page ou les mentions légales récurrentes) pour te concentrer sur le fond du document.\n"
        "4. Réponds de manière professionnelle en utilisant des listes à puces pour la clarté."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

# Initialisation vide au démarrage
qa_chat = None
vectordb = None

In [69]:
def process_pdf(file):
    global qa_chat, vectordb

    qa_chat = None
    vectordb = None

    # Libérer les fichiers sur le disque
    import gc
    gc.collect()

    base_name = os.path.basename(file.name)
    clean_name = re.sub(r'[^\w\s-]', '', base_name.replace(".pdf", "")).strip()
    db_path = f"./chroma_{clean_name}"

    # 1. Charger le document uploadé
    loader = PyPDFLoader(file.name)
    documents = loader.load()

    # 2. Découper en segments
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150, separators = ["\n\n", "\n", " ", ""])
    chunks = text_splitter.split_documents(documents)

    # 3. Créer modèle embeddings
    hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # 4. Créer la base de données en mémoire
    vectordb = Chroma.from_documents(
        documents=chunks,
        embedding=hf_embeddings,
        persist_directory=db_path
    )

    # 5. Mettre à jour le retriever de qa_chat existant
    qa_chat = ConversationalRetrievalChain.from_llm(
        llm=llm,
        memory=memory,
        retriever = vectordb.as_retriever(search_kwargs={"k": 3}),
        combine_docs_chain_kwargs={"prompt": instruction_prompt},
        # forcer l'historique à rester une liste
        get_chat_history=lambda h: h
    )

    return "PDF analysé avec succès !"

In [70]:
def chat_interactif(message, history):
    # 1. Sécurité
    if qa_chat is None:
        history.append([message, "⚠️ Veuillez d'abord charger un PDF."])
        return "", history

    # 2. Lancer la recherche
    result = qa_chat.invoke({"question": message, "chat_history": history})
    reponse_texte = result.get("answer", "")

    # 3. Mettre à jour l'historique au format GRADIO [[user, bot]]
    history.append([message, reponse_texte])

    # 4. Renvoyer le message vide (pour vider la barre de texte) et l'historique
    return "", history

In [71]:
import gradio as gr
import gc

# 1. Fonction de nettoyage (Reset)
def reinitialiser_tout():
    global qa_chat, vectordb, memory
    qa_chat = None
    vectordb = None
    if 'memory' in globals() and memory is not None:
        memory.clear()
    gc.collect()
    return [], "", None, "En attente de document..."

# Fermeture des sessions précédentes pour éviter les bugs
gr.close_all()

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 Mon Expert PDF Intelligent")

    with gr.Row():
        # --- COLONNE DE GAUCHE : OUTILS ---
        with gr.Column(scale=1):
            file_input = gr.File(label="Téléverser votre PDF")
            upload_button = gr.Button("🚀 Analyser le document", variant="primary")
            status_label = gr.Label(value="En attente de document...")

            gr.Markdown("### ⚡ Actions Rapides")
            # BOUTONS ACTION:
            btn_resume = gr.Button("📝 Résumer le document")
            btn_points = gr.Button("🔑 Points clés")

            gr.Markdown("---")
            reset_button = gr.Button("🗑️ Réinitialiser tout", variant="stop")

        # --- COLONNE DE DROITE : CHAT ---
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(type="tuples")
            msg = gr.Textbox(label="Posez votre question sur le document")
            clear = gr.ClearButton([msg, chatbot], value="Effacer l'historique")

    # --- LOGIQUE DES ÉVÉNEMENTS ---

    # Analyse
    upload_button.click(process_pdf, inputs=[file_input], outputs=[status_label])

    # Chat classique (Entrée)
    msg.submit(chat_interactif, [msg, chatbot], [msg, chatbot])

    # ACTION 1 : Résumer
    btn_resume.click(
        fn=lambda h: chat_interactif("Peux-tu me faire un résumé détaillé et structuré de ce document ?", h),
        inputs=[chatbot],
        outputs=[msg, chatbot]
    )

    # ACTION 2 : Points clés
    btn_points.click(
        fn=lambda h: chat_interactif("Quels sont les points clés, les chiffres ou les conclusions importantes de ce document ?", h),
        inputs=[chatbot],
        outputs=[msg, chatbot]
    )

    # Réinitialisation
    reset_button.click(
        fn=reinitialiser_tout,
        inputs=[],
        outputs=[chatbot, msg, file_input, status_label]
    )

# Lancement avec lien public
demo.launch(share=True)

Closing server running on port: 7860


/tmp/ipython-input-3552511052.py:17: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3552511052.py:37: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chatbot = gr.Chatbot(type="tuples")
/tmp/ipython-input-3552511052.py:37: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(type="tuples")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://28a06f850e3c6df236.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
